In [1]:
import Covid19_model as Cm
import time
from read_training import *
import pickle
from Reinforce_learn import *
import pandas as pd
from Offline_test import *

In [2]:
"""model parameters"""
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))/pop
init_state = init_state.tolist()
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
init_state.append([1])
init_state.append([0]*n_site)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.3e6
path = 'result/transferRL_' + str(round(n_site)) + 'low'

In [3]:
"""system parameters"""
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
env = Cm.Env_model(init_state, deter_para, in_deter_para, seed=0)
env.set_para_truth(para_truth)
agent = read_training(path, n_state, n_action, env, 54, 54)


model have been loaded


In [5]:
tic = time.time()
action_store = []
state = env.init_state.copy()
observation = state[:6] * pop
state_buffer = np.zeros([T, 6])
new_buffer = np.zeros([T, 4])
budget, score, ni, ne = env.B, 0, 0, 0 
done = False
n_select = 5
epd = 1
p = np.zeros((1, n_site))
action_buffer = []
reward_buffer = []
index = np.array(range(n_select))
actor = agent[0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    if t > start_time: 
        agent, select_id = train_online(agent, env, index, epd, p_set)
        actor = agent[select_id]
    state_nn = state[0]+state[1]+state[2]+state[3]+state[4]+state[5]+state[6]+state[7]
    if t > start_time:
        act = actor.act_target(np.array(state_nn))[0]
        action = np.reshape(act, (5, n_site))   
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck, done = env.step_online(t, state, observation,
                                                                             action, p_set,
                                                                             n_select, 100)
    score += reward
    ni += new_i
    ne += new_e
    if not done and t > start_time: 
        env.space_update(ck, p_set, n_select)
        index = ck.argsort()[0][:n_select]
    action_store.append(action)
    action = np.reshape(action, (5, n_site))
    state = state[:6] * env.pop
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    budget -= cost
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time()-tic)

 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-229.27
 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-229.27
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2300000.00, reward:-554.35
 Time: 1 | S: 25495797.0, E: 144.0, A: 0.0, I: 47.3, D: 0.0, R: 4.0, Budget: 2300000.00, reward:-554.35
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 

[0.16029180415511637, 0.16118056580432832]
[0.8127771251266731, 0.8137648231073416]
[0.09730674426134979, 0.09733690671753165]
[0.04326035674972256, 0.04345233680222825]
 Time: 10 | S: 25308064.9, E: 367.2, A: 1066.6, I: 1690.7, D: 2.8, R: 184807.8, Budget: 2185808.48, reward:-10841.46
 Time: 10 | S: 25308283.0, E: 356.0, A: 1066.6, I: 1690.7, D: 2.8, R: 184793.0, Budget: 2185808.48, reward:-10841.46
 Time: 10 | new_e:171.6, cum_e:2459.1, new_i:268.0, cum_i:2471.1
[[1.2886641e-09 9.9988484e-01 9.9999988e-01 1.0000000e+00 9.9996579e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999976e-01
  1.0000000e+00 3.1574756e-02 4.5876789e-08 1.5348803e-04 1.0000000e+00
  1.0000000e+00]
 [9.9970347e-01 9.9914837e-01 9.9999464e-01 4.5458536e-04 1.0000000e+00
  1.0000000e+00 9.9999988e-01 9.9999976e-01 1.0000000e+00 9.9999988e-01
  9.9999940e-01 9.9987769e-01 9.9999976e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 9.9999976e-01 9.9999976e-01 1.00000

[0.1606111557998595, 0.16061214694929501]
[0.81374178409805, 0.8137419107266806]
[0.09733379392558802, 0.097333794115559]
[0.04344211144450625, 0.04344211275501735]
 Time: 16 | S: 24979270.5, E: 64.3, A: 823.5, I: 1639.5, D: 4.6, R: 514197.6, Budget: 1965714.09, reward:-12913.48
 Time: 16 | S: 24979446.0, E: 73.0, A: 823.5, I: 1639.5, D: 4.6, R: 514171.0, Budget: 1965714.09, reward:-12913.48
 Time: 16 | new_e:29.6, cum_e:2913.2, new_i:114.9, cum_i:3517.2
[[2.4100268e-04 9.9857175e-01 1.0000000e+00 1.0000000e+00 9.9999976e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 3.5648067e-09 1.4564257e-07 8.6566876e-04 1.0000000e+00
  1.0000000e+00]
 [9.9967134e-01 9.9098444e-01 1.0000000e+00 9.9999774e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 9.9941909e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.9995375e-01 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1

[0.16061153731606861, 0.16061153960601637]
[0.8137419082697882, 0.8137419082932754]
[0.09733379409158248, 0.09733379409158913]
[0.043442112638067874, 0.04344211263806964]
 Time: 22 | S: 24639008.0, E: 15.3, A: 373.1, I: 1075.2, D: 5.7, R: 855522.8, Budget: 1737310.60, reward:-13307.86
 Time: 22 | S: 24639144.0, E: 24.0, A: 373.1, I: 1075.2, D: 5.7, R: 855497.0, Budget: 1737310.60, reward:-13307.86
 Time: 22 | new_e:6.9, cum_e:2989.6, new_i:46.2, cum_i:3934.8
[[2.2452275e-04 9.9999321e-01 9.9998736e-01 9.9999893e-01 1.0000000e+00
  9.9999392e-01 9.9885452e-01 9.9998164e-01 9.9999881e-01 1.0000000e+00
  9.9996591e-01 9.8981726e-01 3.2036585e-11 9.9999905e-01 9.9998915e-01
  1.0000000e+00]
 [9.9492234e-01 9.9999988e-01 9.9999940e-01 9.9999666e-01 9.9999857e-01
  9.9998569e-01 9.9996281e-01 5.5516529e-04 1.0000000e+00 9.9998975e-01
  1.0000000e+00 9.9995208e-01 1.0000000e+00 9.9998927e-01 9.9994147e-01
  9.9669927e-01]
 [9.9999952e-01 9.9998081e-01 9.9660742e-01 9.9999046e-01 9.9995995e-01

[0.16061153808896084, 0.16061153844281648]
[0.8137419082700328, 0.813741908270034]
[0.0973337940915881, 0.09733379409158811]
[0.043442112638069415, 0.04344211263806946]
 Time: 28 | S: 24315176.2, E: 4.5, A: 154.9, I: 657.8, D: 6.8, R: 1179999.9, Budget: 1516837.78, reward:-13457.94
 Time: 28 | S: 24315282.0, E: 3.0, A: 154.9, I: 657.8, D: 6.8, R: 1179975.0, Budget: 1516837.78, reward:-13457.94
 Time: 28 | new_e:2.0, cum_e:3009.8, new_i:18.4, cum_i:4102.3
[[7.25242010e-14 9.99998569e-01 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 8.22439313e-01
  3.02050357e-10 9.99905705e-01 1.00000000e+00 1.00000000e+00]
 [9.99999166e-01 9.99986768e-01 1.00000000e+00 9.99920726e-01
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.01934005e-04
  1.00000000e+00 1.00000000e+00 1.00000000e+00 9.99782622e-01
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e

...

[0.16061153823769703, 0.16061153823769708]
[0.8137419082700331, 0.8137419082700332]
[0.09733379409158811, 0.09733379409158811]
[0.04344211263806946, 0.04344211263806946]
 Time: 62 | S: 23593991.5, E: 0.1, A: 1.3, I: 76.7, D: 9.1, R: 1901921.3, Budget: 536655.93, reward:-13654.39
 Time: 62 | S: 23594106.0, E: 0.0, A: 1.3, I: 76.7, D: 9.1, R: 1901908.0, Budget: 536655.93, reward:-13654.39
 Time: 62 | new_e:0.0, cum_e:3023.9, new_i:0.2, cum_i:4216.9
[[4.55436944e-14 7.76795121e-12 5.33600808e-09 1.62814529e-16
  4.82188398e-07 1.00000000e+00 1.72924876e-04 1.00000000e+00
  2.26785893e-15 6.79439618e-14 9.66063370e-14 2.65249517e-10
  9.99992609e-01 8.07290198e-05 4.23510273e-14 1.00000000e+00]
 [6.16197940e-04 4.44309819e-07 2.66346927e-15 1.43381307e-09
  1.00000000e+00 1.19257817e-14 1.00000000e+00 1.00000000e+00
  9.99999881e-01 1.00000000e+00 7.31018995e-15 9.99568284e-01
  1.00000000e+00 9.99999881e-01 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00

[0.16061153823769703, 0.16061153823769708]
[0.8137419082700331, 0.8137419082700332]
[0.09733379409158811, 0.09733379409158811]
[0.04344211263806946, 0.04344211263806946]
 Time: 67 | S: 23589627.7, E: 0.0, A: 0.7, I: 66.9, D: 9.4, R: 1906295.3, Budget: 397305.04, reward:-13673.81
 Time: 67 | S: 23589735.0, E: 0.0, A: 0.7, I: 66.9, D: 9.4, R: 1906284.0, Budget: 397305.04, reward:-13673.81
 Time: 67 | new_e:0.0, cum_e:3024.0, new_i:0.1, cum_i:4217.4
[[1.4222867e-12 5.0599789e-12 3.7554741e-13 7.6397634e-16 4.3639719e-08
  1.0000000e+00 6.9958951e-08 1.0000000e+00 5.8662889e-15 1.1953433e-13
  2.3954473e-14 2.0277958e-11 1.0000000e+00 1.0845954e-08 2.6984621e-14
  9.9999917e-01]
 [7.3717269e-03 2.2130958e-03 2.2808088e-14 5.5463056e-09 1.0000000e+00
  6.9199644e-14 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01
  4.0829444e-15 9.9999416e-01 1.0000000e+00 1.0000000e+00 1.0000000e+00
  9.9999988e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000

[0.16061153823769703, 0.16061153823769708]
[0.8137419082700331, 0.8137419082700332]
[0.09733379409158811, 0.09733379409158811]
[0.04344211263806946, 0.04344211263806946]
 Time: 73 | S: 23592483.8, E: 0.0, A: 0.3, I: 63.7, D: 9.8, R: 1903442.4, Budget: 249987.86, reward:-13698.47
 Time: 73 | S: 23592593.0, E: 0.0, A: 0.3, I: 63.7, D: 9.8, R: 1903429.0, Budget: 249987.86, reward:-13698.47
 Time: 73 | new_e:0.0, cum_e:3024.1, new_i:0.0, cum_i:4217.8
[[2.80509089e-17 6.56285356e-12 4.94747286e-15 2.37370448e-16
  2.67191741e-10 1.00000000e+00 1.31364963e-09 1.00000000e+00
  6.23431999e-17 6.70089594e-14 8.57590705e-14 2.17458222e-11
  1.00000000e+00 1.39235914e-11 3.60701269e-13 9.99998689e-01]
 [5.21170591e-07 1.62920713e-07 2.06277444e-16 5.24252950e-13
  1.00000000e+00 4.05963283e-15 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 7.81848892e-16 9.99988317e-01
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00

[0.16061153823769703, 0.16061153823769708]
[0.8137419082700331, 0.8137419082700332]
[0.09733379409158811, 0.09733379409158811]
[0.04344211263806946, 0.04344211263806946]
 Time: 79 | S: 23594367.2, E: 0.0, A: 0.2, I: 60.1, D: 10.2, R: 1901562.3, Budget: 88586.73, reward:-13721.78
 Time: 79 | S: 23594477.0, E: 0.0, A: 0.2, I: 60.1, D: 10.2, R: 1901547.0, Budget: 88586.73, reward:-13721.78
 Time: 79 | new_e:0.0, cum_e:3024.1, new_i:0.0, cum_i:4218.0
[[3.03424301e-16 1.02544731e-12 7.76558790e-14 3.68764814e-15
  1.60730194e-13 1.00000000e+00 6.28312981e-14 1.00000000e+00
  1.17055874e-13 1.28474874e-14 3.40999168e-14 5.78613581e-11
  1.00000000e+00 9.48165717e-15 2.01363448e-12 9.99982476e-01]
 [5.14588144e-04 6.66176492e-09 9.09461241e-14 3.11156858e-17
  1.00000000e+00 2.52231265e-12 1.00000000e+00 1.00000000e+00
  9.97892916e-01 1.00000000e+00 3.49298984e-17 3.00021566e-05
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00

 Time: 85 | S: 23624396.6, E: 21.5, A: 0.1, I: 77.5, D: 10.6, R: 1871493.7, Budget: 5655.85, reward:-13921.41
 Time: 85 | S: 23624503.0, E: 15.0, A: 0.1, I: 77.5, D: 10.6, R: 1871480.0, Budget: 5655.85, reward:-13921.41
 Time: 85 | new_e:16.0, cum_e:3064.3, new_i:13.0, cum_i:4249.0
[[5.6422995e-18 1.7540248e-14 3.2626107e-11 7.1240350e-15 1.0763730e-11
  1.0000000e+00 8.2147628e-10 1.0000000e+00 2.7161894e-14 2.0223608e-15
  9.9516376e-15 1.4427874e-08 9.9999964e-01 7.5955009e-10 3.7849051e-14
  9.9989307e-01]
 [6.7629302e-09 1.2580999e-08 5.1535467e-14 5.8805316e-10 1.0000000e+00
  1.5426699e-13 1.0000000e+00 1.0000000e+00 5.6456872e-07 1.0000000e+00
  2.2743285e-17 8.9156842e-09 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.

In [6]:
result_state = pd.DataFrame(state_buffer)
result_state.to_csv(r'TL_low_state_16.csv')
result_new = pd.DataFrame(new_buffer)
result_new.to_csv(r'TL_low_new_16.csv')

In [10]:
import pickle
pickle.dump(action_store, open("TL_low_action.pth", 'wb'))

In [8]:
(time.time()-tic) / ( T - start_time + 1)

71.08530571881462

In [9]:
3225.0+ 4385.0

7610.0